In [59]:
import torch
from torchvision import transforms
from torchvision import datasets
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [53]:
#Defining Transformations
transform = transforms.Compose([
    transforms.Resize((500,500)),
    transforms.ToTensor()
    #transforms.Normalize(mean=[0.5,0.5,0.5], std=[.5,0.5,0.5])
])

negImg = datasets.ImageFolder(root="datasets/sixray/train_data/",transform=transform)


dataloader = torch.utils.data.DataLoader(negImg, batch_size=4, shuffle=False)

# Get the class-to-index mapping
class_to_idx = negImg.class_to_idx

# Initialize a dictionary to store the counts
class_counts = {class_label: 0 for class_label in class_to_idx.values()}

# Iterate over the dataset
for _, class_label in negImg:
    class_counts[class_label] += 1

# Print the counts for each class
for class_label, count in class_counts.items():
    class_name = negImg.classes[class_label]
    print(f"Class '{class_name}': {count} images")


# Initialize variables to store min and max pixel values
min_pixel_value = float('inf')
max_pixel_value = float('-inf')

# Iterate over the dataset
for images, _ in dataloader:
    # Calculate the current min and max pixel values
    current_min = torch.min(images)
    current_max = torch.max(images)

    # Update min and max values if necessary
    min_pixel_value = min(min_pixel_value, current_min)
    max_pixel_value = max(max_pixel_value, current_max)

# Print the min and max pixel values
print("Min pixel value:", min_pixel_value.item())
print("Max pixel value:", max_pixel_value.item())

Class 'normal': 420 images
Min pixel value: 0.0
Max pixel value: 1.0


In [ ]:
# Get a batch of images from the data loader
images, _ = next(iter(dataloader))

# Plot the images
fig, axes = plt.subplots(1, len(images), figsize=(10, 5))

for idx, image in enumerate(images):
    # Convert the image tensor to a NumPy array and transpose the dimensions
    image_np = image.numpy().transpose(1, 2, 0)

    # Clip the values to the valid range [0, 1]
    image_np = np.clip(image_np, 0, 1)
    
    # Plot the image
    axes[idx].imshow(image_np)
    axes[idx].axis('off')


In [ ]:
for images, labels in dataloader:
    # Print the min and max pixel values
    print("Min pixel value:", torch.min(images).item())
    print("Max pixel value:", torch.max(images).item())
    

In [ ]:
class Autoencoder_CNN(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)

        self.encoder = nn.Sequential(
            nn.Conv2d(3,16,3 ,stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2),

            nn.Conv2d(16,8,3, stride=1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2),

            nn.Conv2d(8,8,3, stride=1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(8,8,3 ,stride=1, padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=2, mode='bilinear'),

            nn.ConvTranspose2d(8,16,3, stride=1,padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=2,mode='bilinear'),

            nn.ConvTranspose2d(16,3,3, stride=1,padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=2,mode='bilinear')

        )